In [ ]:
!pip install ../input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4/ > /dev/null # no output
package_path = '../input/unetmodelscript' # add unet script dataset
import sys
sys.path.append(package_path)
from model import Unet # import Unet model from the script

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from tqdm.notebook import tqdm_notebook as tqdm
import seaborn as sns
import albumentations  as albu
from albumentations.pytorch import ToTensor
import random

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision
from torchvision import models
from torch.autograd import Function

In [ ]:
# 乱数のシードを設定
seed = 1234
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# データ読み込み

In [ ]:
input_dir = "../input/severstal-steel-defect-detection/"
input_dir_Train  = os.path.join(input_dir, 'train_images')
input_dir_Test  = os.path.join(input_dir, 'test_images')
filelist_Train = os.listdir(input_dir_Train)
filelist_Test = os.listdir(input_dir_Test)

In [ ]:
image = cv2.imread(os.path.join(input_dir_Test, filelist_Test[0]))
plt.imshow(image)
plt.xticks([])
plt.yticks([])
plt.savefig('image.jpg')
plt.show()

In [ ]:
df_path = os.path.join(input_dir, 'train.csv')
df = pd.read_csv(df_path)
df.head()

In [ ]:
def make_df(df):
    df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
    df['defects'] = df.count(axis=1)
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["defects"], random_state=seed)
    return train_df, val_df

In [ ]:
train_df, val_df = make_df(df)

In [ ]:
test_df_path = os.path.join(input_dir, 'sample_submission.csv')
test_df = pd.read_csv(test_df_path)
test_df.head()

# Data Augmentation

In [ ]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
def get_augmentation(mean, std, phase):
    
    if phase == 'train':
        transform = [
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.Resize(256, 256, interpolation=cv2.INTER_NEAREST, p=1),
            albu.Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    else:
        transform = [
            albu.Resize(256, 256, interpolation=cv2.INTER_NEAREST, p=1),
            albu.Normalize(mean=mean, std=std, p=1),
            ToTensor(),
        ]
    
    return albu.Compose(transform)

# Dataloader作成

In [ ]:
def make_mask(index, df):
    filename = df.iloc[index].name
    labels = df.iloc[index, :4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32)
    for idx, label in enumerate(labels):
        if label is not np.nan:
            mask = np.zeros((256*1600), dtype=np.uint8)
            pixels = label.split(' ')
            pixels = [pixels[i:i+2] for i in range(0, len(pixels), 2)]
            for pixel in pixels:
                pos, le = pixel
                pos, le = int(pos), int(le)
                mask[pos-1:pos+le-1] = 1
            masks[:,:,idx] = mask.reshape(256, 1600, order = 'F')
    return filename, masks

In [ ]:
class ValDataset(torch.utils.data.Dataset):
    def __init__(self, df, input_dir, phase):
        self.df = df
        self.input_dir = input_dir
        self.transforms = get_augmentation(mean, std, phase) 
        self.phase = phase
    def __getitem__(self, idx):
        filename, mask = make_mask(idx, self.df)
        image = cv2.imread(os.path.join(self.input_dir, filename))
        augmented = self.transforms(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']
        mask = mask[0].permute(2, 0, 1)
        return image, mask
    def __len__(self):
        return len(self.df)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, input_dir, df, phase):
        self.input_dir = input_dir
        self.fnames = df['ImageId'].unique().tolist()
        self.transforms = get_augmentation(mean, std, phase)
        self.phase = phase
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        image = cv2.imread(os.path.join(self.input_dir, fname))
        augmented = self.transforms(image=image)
        image = augmented['image']
        return fname, image
    def __len__(self):
        return len(self.fnames)

In [ ]:
val_dataset = ValDataset(val_df, input_dir_Train, phase = 'val')

# 動作確認
index = 0
image, mask = val_dataset.__getitem__(index) 
print(image.size())
plt.imshow(image.to('cpu').detach().numpy().copy()[0])
plt.show()
print(mask.size())
plt.imshow(mask.to('cpu').detach().numpy().copy()[2])
plt.show()

In [ ]:
test_dataset = TestDataset(input_dir_Test, test_df, phase = 'test')

# 動作確認
index = 0
fname, image = test_dataset.__getitem__(index) 
print('filename : {}'.format(fname))
print(image.size())
plt.imshow(image.to('cpu').detach().numpy().copy()[0])
plt.show()

In [ ]:
batch_size = 4

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=6)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)

# 動作確認
batch_iterator = iter(test_dataloader)  # イテレータに変換
fname, inputs = next(
    batch_iterator)  # 1番目の要素を取り出す
print(inputs.size())

# ネットワークを用意

In [ ]:
resnet18 = Unet("resnet18", encoder_weights="imagenet", classes=4, activation=None)
resnet34 = Unet("resnet34", encoder_weights="imagenet", classes=4, activation=None)
resnet50 = Unet("resnet50", encoder_weights="imagenet", classes=4, activation=None)

resnet18.load_state_dict(torch.load("../input/resnet-weights/resnet18_CP18.pth"))
resnet34.load_state_dict(torch.load("../input/resnet-weights/resnet34_CP19.pth"))
resnet50.load_state_dict(torch.load("../input/resnet-weights/resnet50_CP16.pth"))

print('ネットワーク設定完了：学習済みの重みをロードしました')

# 推論を実行

In [ ]:
def dice_coeff(pred, mask):
    with torch.no_grad():
        batch_size = len(pred)
        pred = pred.view(batch_size, -1) # Flatten
        mask = mask.view(batch_size, -1)  # Flatten
        pred = (pred>0.5).float()
        mask = (mask>0.5).float()
        smooth = 0.0001
        intersection = (pred * mask).sum()
        dice_pos = (2. * intersection + smooth) / (pred.sum() + mask.sum() + smooth) 
        intersection = ((pred + mask) == 0).sum()
        dice_neg = (2. * intersection + smooth) / ((pred == 0).sum() + (mask == 0).sum() + smooth)
        dice = (dice_pos + dice_neg) / 2.0
        return dice.item()

In [ ]:
# Validate one model
def validate(model, dataloader):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    model = model.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    
    num_val_imgs = len(val_dataloader.dataset)
    batch_size = val_dataloader.batch_size
    
    epoch_acc = 0.0
    with torch.no_grad():
        for img, mask in tqdm(val_dataloader):
            
            model.eval()
            
            # GPUが使えるならGPUにデータを送る
            img = img.to(device)
            mask = mask.to(device)

            output = model(img)
                
            prob = torch.sigmoid(output)
            prob = prob.to('cpu').detach()
            mask = mask.to('cpu').detach()
            
            # ダイス係数の合計を更新
            epoch_acc += dice_coeff(prob, mask)
    print('Accuracy : {}'.format(epoch_acc / num_val_imgs * batch_size))
        

In [ ]:
# Validate ensumble model
def validate_ensumble(model1, model2, model3, val_dataloader):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    model1 = model1.to(device)
    model2 = model2.to(device)
    model3 = model3.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    
    num_val_imgs = len(val_dataloader.dataset)
    batch_size = val_dataloader.batch_size
    
    epoch_acc = 0.0
    with torch.no_grad():
        for img, mask in tqdm(val_dataloader):
            
            model1.eval()
            model2.eval()
            model3.eval()
            
            # GPUが使えるならGPUにデータを送る
            img = img.to(device)
            mask = mask.to(device)

            output1 = model1(img)
            output2 = model2(img)
            output3 = model3(img)
            
            prob1 = torch.sigmoid(output1)
            prob2 = torch.sigmoid(output2)
            prob3 = torch.sigmoid(output3)
            
            prob = (prob1 + prob2 + prob3) / 3.0
            
            prob = prob.to('cpu').detach()
            mask = mask.to('cpu').detach()
            
            # ダイス係数の合計を更新
            epoch_acc += dice_coeff(prob, mask)
    print('Accuracy : {}'.format(epoch_acc / num_val_imgs * batch_size))
        

In [ ]:
# Resnet18 Validation
validate(resnet18, val_dataloader)

In [ ]:
# Resnet34 Validation
validate(resnet34, val_dataloader)

In [ ]:
# Resnet50 Validation
validate(resnet50, val_dataloader)

In [ ]:
# Resnet Ensumble Validation
validate_ensumble(resnet18, resnet34, resnet50, val_dataloader)

In [ ]:
# Show and Save predicted image
def predict_show(index, test_dataloader, model, spath=None):
    
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    model = model.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    
    with torch.no_grad():
        
        model.eval()
        
        fname, img = test_dataloader.dataset.__getitem__(index)
        
        img = img.unsqueeze(0)
        
        # GPUが使えるならGPUにデータを送る
        img = img.to(device)
            
        output = model(img)
            
        prob = torch.sigmoid(output)
        prob = (prob>0.5).float()
            
        img = img.to('cpu').detach().numpy().copy()[0,0]
        prob = prob.to('cpu').detach().numpy().copy()[0,2]
            
        print("元画像")
        print("filename : {}".format(fname))
        plt.imshow(cv2.resize(img, (1600, 256), interpolation=cv2.INTER_NEAREST))
        plt.xticks([])
        plt.yticks([])
        plt.show()
        print("予測画像")
        plt.imshow(cv2.resize(prob, (1600, 256), interpolation=cv2.INTER_NEAREST))
        plt.xticks([])
        plt.yticks([])
        if spath is not None:
            save_path = os.path.join(spath, '{}_pred.jpg'.format(fname[:-4]))
            plt.savefig(save_path)
        plt.show()

In [ ]:
save_path  = "./predicts"

if os.path.exists(save_path) == False:
    os.makedirs(save_path)

index = 0
predict_show(index, test_dataloader, resnet34, save_path)

In [ ]:
def mask2rle(img):
    
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [ ]:
def post_process(pred, min_size):
    
    mask = cv2.resize(pred, (1600, 256), interpolation=cv2.INTER_NEAREST)
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [ ]:
# 初期設定
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# ネットワークをGPUへ
model = resnet34.to(device)

# ネットワークがある程度固定であれば、高速化させる
torch.backends.cudnn.benchmark = True    

min_size = 3500

predictions = []
for i, batch in enumerate(tqdm(test_dataloader)):
    fnames, images = batch
    batch_preds = torch.sigmoid(model(images.to(device)))
    batch_preds = (batch_preds>0.5).float()
    batch_preds = batch_preds.detach().cpu().numpy()
    for fname, preds in zip(fnames, batch_preds):
        for cls, pred in enumerate(preds):
            pred, num = post_process(pred, min_size)
            rle = mask2rle(pred)
            if rle != '':
                predictions.append([fname, rle, cls+1])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['ImageId', 'EncodedPixels', 'ClassId'])
df.to_csv("submission.csv", index=False)